<a href="https://colab.research.google.com/github/kul-pat/Public-Machine-learning/blob/main/keras_tuning_for_heart_disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Tensorflow

In [ ]:
!pip install keras-tuner -q

In [ ]:
import numpy as np
import pandas as pd
from kerastuner.tuners import RandomSearch
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam,rmsprop
from keras.layers import Dropout
from keras import regularizers

ImportError: ignored

In [ ]:
df = pd.read_csv('heart_failure_clinical_records_dataset.csv')
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [ ]:
df.shape

(299, 13)

In [ ]:
X = np.array(df.drop(['DEATH_EVENT'], 1))
y = np.array(df['DEATH_EVENT'])
X

array([[7.500e+01, 0.000e+00, 5.820e+02, ..., 1.000e+00, 0.000e+00,
        4.000e+00],
       [5.500e+01, 0.000e+00, 7.861e+03, ..., 1.000e+00, 0.000e+00,
        6.000e+00],
       [6.500e+01, 0.000e+00, 1.460e+02, ..., 1.000e+00, 1.000e+00,
        7.000e+00],
       ...,
       [4.500e+01, 0.000e+00, 2.060e+03, ..., 0.000e+00, 0.000e+00,
        2.780e+02],
       [4.500e+01, 0.000e+00, 2.413e+03, ..., 1.000e+00, 1.000e+00,
        2.800e+02],
       [5.000e+01, 0.000e+00, 1.960e+02, ..., 1.000e+00, 1.000e+00,
        2.850e+02]])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.15,random_state=0)

In [ ]:
# convert the data to categorical labels
from keras.utils.np_utils import to_categorical

Y_train = to_categorical(y_train, num_classes=None)
Y_test = to_categorical(y_test, num_classes=None)
print (Y_train.shape)
print (Y_train[:10])

(254, 2)
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]]


In [ ]:
# convert into binary classification problem - heart disease or no heart disease
Y_train_binary = y_train.copy()
Y_test_binary = y_test.copy()

Y_train_binary[Y_train_binary > 0] = 1
Y_test_binary[Y_test_binary > 0] = 1

print(Y_train_binary[:20])

[0 0 0 1 1 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0]


In [ ]:
def build_model(hp):
  model = keras.Sequential()
  for i in range(hp.Int('num_layer',2,40)):
      model.add(layers.Dense(units=hp.Int("units_"+str(i), min_value= 8, max_value = 512,step= 32),
                             activation="relu",kernel_initializer='normal'))
      model.add(layers.Dense(1, activation="sigmoid"))
      model.compile(
        optimizer=tensorflow.keras.optimizers.Adam(
            hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
        ),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )
  return model


In [ ]:
tuner = RandomSearch(
                     build_model,
                     objective="val_accuracy",
                     max_trials=3,
                     executions_per_trial=2,
                     overwrite=True,
                     directory="my_dir",
                     project_name="helloworld")

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layer (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 40, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 8, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 8, 'max_value': 512, 'step': 32, 'sampling': None}


In [ ]:
tuner.search(X_train, y_train, epochs=2, validation_data=(X_test, y_test))

Trial 3 Complete [00h 00m 09s]
val_accuracy: 0.6000000238418579

Best val_accuracy So Far: 0.6000000238418579
Total elapsed time: 00h 00m 19s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in my_dir/helloworld
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_layer: 37
units: 296
learning_rate: 0.001
Score: 0.6000000238418579
Trial summary
Hyperparameters:
num_layer: 30
units: 72
learning_rate: 0.001
Score: 0.6000000238418579
Trial summary
Hyperparameters:
num_layer: 14
units: 392
learning_rate: 0.001
Score: 0.6000000238418579
Trial summary
Hyperparameters:
num_layer: 9
units: 232
learning_rate: 0.01
Score: 0.6000000238418579
Trial summary
Hyperparameters:
num_layer: 3
units: 360
learning_rate: 0.0001
Score: 0.5333333512147268
